In [ ]:
from __future__ import print_function
from keras import layers 
from keras import models

from keras.models import Sequential
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Flatten, Dense, Activation, Dropout
from keras.preprocessing import image
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import cv2 

import matplotlib.pyplot as plt

import os
import os.path
import random 
from skimage.color import rgb2gray

# Project: Building Convultional Neural Network using Tensorflow and Keras that detects blurry spots in SEM images 


In [ ]:
#Load DataSet
train_focus_path = "Testing_Focus_Images/"
train_outfocus_path = "Testing_Outfocus_Images/"

training_set = []
labels = []
count = 0

# Image Processing Functions 


In [ ]:
#The following code reads the image and performs laplacian of gaussian computer vision algorithm. 
#The graph is also created as a part of the function. 
from PIL import Image

def check_shape(image, label):
    image = cv2.imread(image)
    height, width, channels = image.shape 

    if (height*width > 330000000):
        return large_classify(image, 20, label)
    else:
        gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        _,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)
    
        contours,hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnt = contours[0]
        x,y,w,h = cv2.boundingRect(cnt)
    
        image = image[y:y+h,x:x+w]
        return small_classify(image, 20, label)

def splice_image(image, x, y):
    test = image
    my_slice = test[x:y, x:y]
    return my_slice

#Pick random number of positions in image and detects blobs 
def large_classify(image, num, label):
    image = cv2.resize(image, dsize=(10000, 10000))
    #height, width, channels = image.shape
    placer = num - 1
    X = 100
    Y = 120
    image = rgb2gray(image)
    for i in range(placer):
        try:
            im = splice_image(image, X, Y)
            im = cv2.resize(im, dsize=(800, 800))
            #im = splice_image(im, 1, 200)
            #im = cv2.resize(im, dsize=(64, 64))
            rotate = random.randint(0, 360)
            im = im.rotate(rotate, expand=True)
            training_set.append(im)
            labels.append(label)
            X = X+100
            Y = Y+100
        except: 
            continue

def small_classify(image, num, label):
    height, width, channels = image.shape
    placer = num - 1
    X = 1
    Y = 900
    image = rgb2gray(image)
    for i in range(placer):
        try:
            im = splice_image(image, X, Y)
            im = cv2.resize(im, dsize=(800, 800))
            rotate = random.randint(0, 360)
            im = im.rotate(rotate, expand=True)
            training_set.append(im)
            labels.append(label)
            X = X+900
            Y = Y+900
        except -215:
            print("skip")
            continue

# Loading Images to Dataset

In [ ]:
#Load Train Data 
i = 1
for filename in os.listdir(train_focus_path):
    print(str(i)+".) " + filename)
    imagepath = train_focus_path + filename
    check_shape(imagepath, 1)
    i = i+1

In [ ]:
i = 1
for filename in os.listdir(train_outfocus_path):
    print(str(i)+".) " + filename)
    imagepath = train_outfocus_path + filename
    check_shape(imagepath, 0) 
    i = i+1

In [ ]:
training_set

In [ ]:
labels

In [ ]:
#creating validation data 
train_set_stacked = np.stack(training_set)
train_set_norm = train_set_stacked/255.
label_cat = to_categorical(labels)
X_train, X_test, y_train, y_test = train_test_split(train_set_norm, label_cat, train_size=0.5, random_state=42)

X_train = X_train.reshape(-1,800, 800, 1)
X_test = X_test.reshape(-1,800, 800, 1)

In [ ]:
y_train.shape

In [ ]:
label_cat.shape

In [ ]:
train_set_norm.shape


In [ ]:
X_train.shape

In [ ]:
# My LeNet architecture
model = Sequential()

# conv filters of 5x5 each

# Layer 1
model.add(Convolution2D(200, (10,10), input_shape=(800, 800, 1), data_format='channels_last'))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(10, 10)))  

# Layer 2
model.add(Convolution2D(150, 10, 10))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(5, 5)))

# Layer 3
model.add(Convolution2D(100, 5, 5))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2))) 
 

model.add(Flatten())

# Layer 4 
model.add(Dense(10))
model.add(Dense(5))


model.add(Dense(2))

# Layer 5
model.add(Activation("sigmoid"))

In [ ]:
from keras import optimizers
sgd = optimizers.SGD(lr=0.08, decay=1e-6, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=6, validation_data=(X_test, y_test))